<a href="https://colab.research.google.com/github/aneeshoberoi/churnhw/blob/main/ChurnNoteBook_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [418]:
import pandas as pd
import numpy as np
from collections import OrderedDict

#pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.options.display.float_format = "{:,.2f}".format
pd.set_option("display.precision",2)

In [382]:
source='https://raw.githubusercontent.com/aneeshoberoi/churnhw/main/churn_homework%20-%20homework_orders.csv?token=AP7KM2QHHOD4JIF5HIHN4M27SWP6C'
raw=pd.read_csv(source, sep=",")

In [ ]:
#raw=df.head(100)
#raw.set_index('order_id')

In [ ]:
# assigning to working copy for indicator variables

# **Basic Statistics**

In [383]:
raw.customer.nunique()

10316

In [384]:
raw.restaurant_total.sum()

1372249.61

In [385]:
raw.order_id.nunique()

42198

# ***Data Cleaning***

In [395]:
# assigning to working copy for indicator variables
test=raw

In [396]:
# Remove zero dollar txns
test=test[test['restaurant_total']>0]

In [388]:
test.customer.nunique()

10316

In [389]:
test.order_id.nunique()

41911

In [390]:
test.restaurant_total.sum()

1372252.61

In [398]:
# Variable Conversion
test['date_purchased']=pd.to_datetime(test['date_purchased'])
test['dateonly_purchased']=test['date_purchased'].dt.date
test['houronly_purchased']=test['date_purchased'].dt.hour
test['datehour_purchased']=test['dateonly_purchased'].astype(str) + "-" + test['houronly_purchased'].astype(str)

test['restaurant_total']=test['restaurant_total'].astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [399]:
# Removing Duplicate Txns - i.e. those made within the same hour at the same shop by a given customer. Keeping last txn only.
test=test.drop_duplicates(subset=['customer', 'shops_id', 'datehour_purchased'], keep='last')


In [400]:
test.customer.nunique()

10316

In [401]:
test.order_id.nunique()

39885

In [402]:
test.restaurant_total.sum()

1301534.06

# ***Data Prep - Indicators***

In [403]:
 #Enumerating Purchases
 test.sort_values(by=['customer','date_purchased'])
 
test = test.assign(order_enum=test.groupby('customer')['datehour_purchased'].cumcount()+1)
test['total_order']=test.groupby('customer')['order_id'].transform('count')

In [404]:
test['bin_orders']=np.where(test['total_order']<7,test['order_enum'],7)

In [405]:
# Calculating Journey Data

test['time_between_purch']=test.groupby(['customer'])['date_purchased'].diff()
test['spend_growth']=test.groupby(['customer'])['restaurant_total'].diff()


# Preparing Zip-Cust Level Variables




In [420]:
#Assigning raw dataset to working copy for this section
shopszip=test[['postal_code', 'shops_id','customer','total_order']].set_index('postal_code')
shopszip=shopszip.drop_duplicates(subset=['customer', 'shops_id'], keep='last')

In [421]:
# calculating shops and customers per postal code
shopszip['shops_per_zip']=shopszip.groupby('postal_code').shops_id.nunique()
shopszip['cust_per_zip']=shopszip.groupby('postal_code').customer.nunique()
shopszip['shops_per_cust_per_zip']=shopszip['shops_per_zip']/shopszip['cust_per_zip']

In [422]:
# Of customers placing orders in a given zip, what # and  % have placed more than 2 orders in total?
temp=shopszip[shopszip['total_order']>1]
shopszip['zip_cust_2_more']=temp.groupby('postal_code')['customer'].count()
shopszip['zip_pct_cust_2_more']=shopszip['zip_cust_2_more']*100/shopszip['cust_per_zip']

# ***Preparing Shop-Cust Level Variables***

In [133]:
#Total customers served by each shop

In [424]:
shopscust=shopszip.set_index('shops_id')
shopscust=shopscust.drop(columns=['shops_per_zip','cust_per_zip','zip_cust_2_more', 'zip_pct_cust_2_more','shops_per_cust_per_zip'] )

# How many customers have placed at least one order at a given shop?

shopscust['cust_per_shop']=shopscust.groupby('shops_id').customer.nunique()

In [425]:
# Of customers placing orders at a given shop, what # and % have placed > 2 total orders/
t=shopscust[shopscust['total_order']>1]
shopscust['shop_cust_2_more']=t.groupby('shops_id')['customer'].count()
shopscust['shop_pct_cust_2_more']=shopscust['shop_cust_2_more']*100/shopscust['cust_per_shop']

In [427]:
shopscust.head(10)

,customer,total_order,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more
shops_id,,,,,
64392,"15,557,129,447.00",5.00,3,3.00,100.00
34911,"18,529,185,191.00",1.00,8,6.00,75.00
24807,"15,315,600,200.00",1.00,1,nan,nan
13989,"13,306,584,713.00",1.00,1,nan,nan
44211,"15,487,909,931.00",1.00,4,2.00,50.00
43299,"6,041,663,939.00",1.00,1,nan,nan
30378,"10,435,705,412.00",2.00,3,3.00,100.00
72708,"8,037,565,352.00",2.00,2,2.00,100.00
48780,"14,548,429,994.00",1.00,7,3.00,42.86


# ***Preparing Shop-Sales Level Variables***

In [432]:
shopsales=test[['shops_id','customer','order_id','restaurant_total']] 

In [433]:
shopsales['total_shop_sales']=shopsales.groupby('shops_id').restaurant_total.transform('sum')
shopsales['total_shop_orders']=shopsales.groupby('shops_id').order_id.transform('count')
shopsales['total_shop_cust_base']=shopsales.groupby('shops_id').customer.transform('nunique')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
shopsales=shopsales.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','order_id','restaurant_total'])

In [440]:
shopsales['sales_qunitiles']=pd.qcut(shopsales['total_shop_sales'],5,labels=False)
shopsales['order_qunitiles']=pd.qcut(shopsales['total_shop_orders'].rank(method='first'),5,labels=False)

#pd.qcut(df['a'].rank(method='first'), 3, labels=False)

# ***Combining to create final analytical dataset***

In [445]:
shopscust_final=shopscust.reset_index()
shopscust_final=shopscust_final.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')


In [446]:
shopscust_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6061 entries, 24807 to 18948
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cust_per_shop         6061 non-null   int64  
 1   shop_cust_2_more      5213 non-null   float64
 2   shop_pct_cust_2_more  5213 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 189.4 KB


In [447]:
# Merging shopscust with original dataset

test2=test.merge(shopscust_final,left_on='shops_id',right_on='shops_id') 


In [ ]:
test2[test2['customer']==6196987925]

In [449]:
# Merging shopszip with original dataset
shopszip_final=shopszip.reset_index()
shopszip_final=shopszip_final.drop_duplicates(subset=['postal_code'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')
shopszip_final.info()
#shopscustzip_final=shopscustzip.drop(columns=['customer','total_order'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3397 entries, 35952 to 18948
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   postal_code             3397 non-null   int64  
 1   shops_per_zip           3397 non-null   int64  
 2   cust_per_zip            3397 non-null   int64  
 3   shops_per_cust_per_zip  3397 non-null   float64
 4   zip_cust_2_more         3005 non-null   float64
 5   zip_pct_cust_2_more     3005 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 185.8 KB


In [450]:
shopszip_final.set_index('postal_code')
test3=test2.merge(shopszip_final,left_on='postal_code',right_on='postal_code') 

In [451]:
# Merging shopsales with original dataset
test4=test3.merge(shopsales,left_on='shops_id',right_on='shops_id')

In [ ]:
test4[test4['customer']==6196987925]

In [ ]:
## add more data operation statements here if needed

In [ ]:
##pending: 
  # promo in first order
  # entry channel
  # shop density (shop per zip)
  # shop loyalty index

# ***Exporting Dataset for usage in Tableau***

In [453]:
test4.to_csv('fin_an_dataset.csv',index=False)

# ***Define Cohorts Based on First Order***